In [44]:
from IPython.display import display, SVG
from typing import Optional, Sequence
import dataclasses
import math

from typesetting import ParagraphItem, ParagraphItemType, Font, Glyph

Source:

http://www.eprg.org/G53DOC/pdfs/knuth-plass-breaking.pdf

In [45]:
from faker import Faker
import textwrap
import hyphen
import uniseg.wordbreak

SOFT_HYPHEN = "\N{SOFT HYPHEN}"
def hyphenate(text: str):
    hyphenator = hyphen.Hyphenator()
    words = []
    for word in uniseg.wordbreak.words(text):
        syllables = hyphenator.syllables(word) if len(word) < 100 else [word]
        if "".join(syllables) == word:
            words.append(SOFT_HYPHEN.join(syllables))
        else:
            words.append(word)
    return "".join(words)

Faker.seed(0xdeadbeef)
lorem = hyphenate("".join([
    "Difficult Quick Foxes with 1034 boxes. ",
    Faker(["la"]).paragraph(50)
]))
print(f"Generated lorem ipsum:\n\n{textwrap.fill(lorem, 72)}")
print(f"\nRepr of first part:\n{textwrap.shorten(lorem, 72)!r}")

Generated lorem ipsum:

Dif­fi­cult Quick Foxes with 1034 boxes. Iusto no­bis in­cidunt nos­trum
quas oc­cae­cati. Ex fa­cilis pariatur quam rep­re­hen­derit fugit
magni. Eius do­lo­rum do­lo­rum maxime hic. Il­lum nemo neque
con­secte­tur volup­ta­tum per­spi­ci­atis ab. Op­tio mo­les­tias no­bis
er­ror facere libero. Harum ip­sam na­tus rep­re­hen­derit do­lore
as­sumenda unde. Tene­tur ab cor­rupti rem modi quia fuga. Ali­quam
pariatur do­lores esse con­secte­tur ipsa fugit. Beatae per­fer­endis
alias deb­itis. Eius mi­nus volup­tas amet quis quae. Fuga cor­rupti
lau­dan­tium in­cidunt in ipsa tem­pora. Unde ver­i­tatis quia iure
volup­tatem fugit culpa. Min­ima ex­cep­turi saepe vero sit of­fi­cia.
At eum rep­re­hen­derit ver­i­tatis quos. Neque pos­simus mo­les­tiae
ver­i­tatis mo­les­tias autem fu­giat min­ima. Lau­dan­tium sapi­ente
alias illo to­tam. Of­fi­cia sed so­luta harum in­cidunt. Nemo
volup­tat­i­bus mag­nam ducimus la­bore of­ficiis. Earum modi sunt
ex­pli­cabo to­tam

In [46]:
rm_face_path = "EBGaramond-VariableFont_wght.ttf"
rm_size = 12
rm_font = Font(path=rm_face_path, em_size=[rm_size, rm_size], features=["onum"])

def text_width(font, text):
    return sum(g.x_advance for g in font.shape(text))

rm_hyphen_width = text_width(rm_font, "-")
rm_space_width = text_width(rm_font, " ")
print(rm_space_width, rm_hyphen_width)

2.4000000000000004 3.2880000000000003


In [47]:
import uniseg.linebreak
print(textwrap.fill("|".join(uniseg.linebreak.line_break_units(lorem)), 72))

Dif­|fi­|cult |Quick |Foxes |with |1034 |boxes. |Iusto |no­|bis
|in­|cidunt |nos­|trum |quas |oc­|cae­|cati. |Ex |fa­|cilis |pariatur
|quam |rep­|re­|hen­|derit |fugit |magni. |Eius |do­|lo­|rum
|do­|lo­|rum |maxime |hic. |Il­|lum |nemo |neque |con­|secte­|tur
|volup­|ta­|tum |per­|spi­|ci­|atis |ab. |Op­|tio |mo­|les­|tias
|no­|bis |er­|ror |facere |libero. |Harum |ip­|sam |na­|tus
|rep­|re­|hen­|derit |do­|lore |as­|sumenda |unde. |Tene­|tur |ab
|cor­|rupti |rem |modi |quia |fuga. |Ali­|quam |pariatur |do­|lores
|esse |con­|secte­|tur |ipsa |fugit. |Beatae |per­|fer­|endis |alias
|deb­|itis. |Eius |mi­|nus |volup­|tas |amet |quis |quae. |Fuga
|cor­|rupti |lau­|dan­|tium |in­|cidunt |in |ipsa |tem­|pora. |Unde
|ver­|i­|tatis |quia |iure |volup­|tatem |fugit |culpa. |Min­|ima
|ex­|cep­|turi |saepe |vero |sit |of­|fi­|cia. |At |eum
|rep­|re­|hen­|derit |ver­|i­|tatis |quos. |Neque |pos­|simus
|mo­|les­|tiae |ver­|i­|tatis |mo­|les­|tias |autem |fu­|giat |min­|ima.
|Lau­|dan­|tium |sapi­

In [48]:
import math

SOFT_HYPHEN_PENALTY=50
INFINITY = 1000  # penalties behyong this are viewed as "infinite".


para_items: list[ParagraphItem] = []
running_width: float = 0.0
running_stems: list[str] = []
for lb_item in uniseg.linebreak.line_break_units(lorem):
    stem = lb_item.rstrip(f" {SOFT_HYPHEN}")
    if len(stem) > 0:
        running_stems.append(stem)
        stem_width = text_width(rm_font, "".join(running_stems)) - running_width
        running_width += stem_width
        para_items.append(ParagraphItem(
            item_type=ParagraphItemType.BOX,
            width=stem_width,
            text=stem,
        ))
    else:
        running_stems, running_width = [], 0.0
    if lb_item.endswith(SOFT_HYPHEN):
        para_items.append(ParagraphItem(
            item_type=ParagraphItemType.PENALTY,
            width=rm_hyphen_width,
            penalty=SOFT_HYPHEN_PENALTY,
            flagged=True,
            text="-",
        ))
    elif lb_item.endswith(" "):
        para_items.append(ParagraphItem(
            item_type=ParagraphItemType.GLUE,
            width=rm_space_width,
            shrinkability=0.5*rm_space_width,
            stretchability=2.0*rm_space_width,
            text=" ",
        ))

# Add finishing glue and forced break
para_items.append(ParagraphItem(
    item_type=ParagraphItemType.GLUE,
    stretchability=INFINITY,
))
para_items.append(ParagraphItem(
    item_type=ParagraphItemType.PENALTY,
    penalty=-INFINITY,
    flagged=True,
))

In [49]:
if False:
    print("First few items:")
    print("\n".join(textwrap.indent(f"{item!r}", "  ") for item in para_items[:10]))
    print("Final few items:")
    print("\n".join(textwrap.indent(f"{item!r}", "  ") for item in para_items[-10:]))

In [50]:
from collections import namedtuple

RunningSum = namedtuple("RunningSum", ["width", "shrinkability", "stretchability"])
def para_item_running_sums(para_items: Sequence[ParagraphItem]) -> Sequence[RunningSum]:
    previous_sum = RunningSum(0, 0, 0)
    sums = [previous_sum]
    for item in para_items:
        if item.item_type != ParagraphItemType.PENALTY:
            previous_sum = RunningSum(
                width=previous_sum.width + item.width,
                shrinkability=previous_sum.shrinkability + item.shrinkability,
                stretchability=previous_sum.stretchability + item.stretchability
            )
        sums.append(previous_sum)
    return sums

In [51]:
def greedy_breaks(para_items: Sequence[ParagraphItem], width: float) -> Sequence[int]:
    "Return sequence of indices in para_items for line breaks."
    sums = para_item_running_sums(para_items)
    current_start_idx = 0
    line_end_indxs = []
    for item_idx, item in enumerate(para_items):
        # Don't break lines at boxes.
        if item.item_type == ParagraphItemType.BOX:
            continue

        # Forced break
        if item.penalty <= -INFINITY:
            line_end_indxs.append(item_idx)
            current_start_idx = item_idx + 1
            continue

        # Compute natural width and total stretch/shrinkability
        line_width = sums[item_idx].width - sums[current_start_idx].width
        if item.item_type == ParagraphItemType.PENALTY:
            line_width += item.width
        line_shrinkability = sums[item_idx].shrinkability - sums[current_start_idx].shrinkability
        line_stretchability = sums[item_idx].stretchability - sums[current_start_idx].stretchability

        if line_width + line_stretchability < width:
            continue

        line_end_indxs.append(item_idx)
        current_start_idx = item_idx + 1
    return line_end_indxs

In [52]:
paper_size = (5 * 72, 5 * 72)
line_width = paper_size[0] - rm_size * (2 + 3)
line_break_idxs = greedy_breaks(para_items, line_width)

In [53]:
from typesetting.cairo import create_cairo_font_face_for_ft_face

ModuleNotFoundError: No module named 'cairo'

In [41]:
import io
from IPython.display import display, SVG
import cffi

svg_file = io.BytesIO()
surface = cairo.SVGSurface(svg_file, paper_size[0], paper_size[1])
ctx = cairo.Context(surface)
with ctx:
    ctx.set_source_rgb(1, 1, 1)
    ctx.rectangle(0, 0, paper_size[0], paper_size[1])
    ctx.fill()
    ctx.set_source_rgb(0, 0, 0)
    ctx.set_font_face(cairo_rm_face)
surface.finish()

display(SVG(svg_file.getvalue()))

TypeError: SVGSurface.create() takes 3 positional arguments but 4 were given

In [29]:
from fpdf import FPDF
def make_pdf(*args, **kwargs):
    pdf = FPDF(*args, orientation="P", unit="pt", format="A4", **kwargs)
    pdf.add_font(family="Roman", fname=rm_face_path)
    pdf.set_font("Roman", size=rm_size)
    pdf.set_text_shaping(use_shaping_engine=True, features={"kern": True, "liga": True, "onum": True})
    return pdf

pdf = make_pdf()
pdf.add_page()

line_start_idx = 0
line_height = rm_size * 1.2
for line_idx, line_end_idx in enumerate(line_break_idxs):
    y = line_height*(1+line_idx)
    line_items = []
    x = rm_size
    running_text, running_width = [], 0.0

    # What delta needs to be taken up by glue
    natural_width = 0.0
    total_stretch, total_shink = 0.0, 0.0
    for item_idx in range(line_start_idx, line_end_idx+1):
        item = para_items[item_idx]
        if item_idx != line_end_idx and item.item_type == ParagraphItemType.PENALTY:
            continue
        if item_idx == line_end_idx and item.item_type == ParagraphItemType.GLUE:
            continue
        natural_width += item.width
        total_stretch += item.stretchability
        total_shink = item.shrinkability
    delta = line_width - natural_width

    # Total slack for delta
    if delta > 0.0:
        adjustment_ratio = delta / total_stretch
    else:
        adjustment_ratio = delta / total_shrink
    
    for item_idx in range(line_start_idx, line_end_idx+1):
        item = para_items[item_idx]
        if item_idx != line_end_idx and item.item_type == ParagraphItemType.PENALTY:
            continue
        if item_idx == line_end_idx and item.item_type == ParagraphItemType.GLUE:
            continue
        running_width += item.width
        if item.item_type == ParagraphItemType.GLUE:
            text = "".join(running_text)
            if text != "":
                pdf.text(x=x, y=y, txt=text)
            x += running_width
            if adjustment_ratio > 0:
                x += item.stretchability * adjustment_ratio
            else:
                x += item.shrinkability * adjustment_ratio

            running_text, running_width = [], 0.0
        else:
            running_text.append(item.text)
    text = "".join(running_text)
    if text != "":
        pdf.text(x=x, y=y, txt=text)
    line_start_idx = line_end_idx+1
pdf.output("example.pdf")